In [1]:
%load_ext autoreload
%autoreload 2

from fastai.text.all import *
from fastai.vision.all import *
import pandas as pd
import torch

In [2]:
OUT_DIM=6

In [3]:
seed = 42

# python RNG
import random
random.seed(seed)

# pytorch RNGs
import torch
torch.manual_seed(seed)
torch.backends.cudnn.deterministic = True
if torch.cuda.is_available(): torch.cuda.manual_seed_all(seed)

# numpy RNG
import numpy as np
np.random.seed(seed)

In [4]:
path = Path("./activations")

In [5]:
files = get_files(path); files

(#359882) [Path('activations/valid/agravo_em_recurso_extraordinario/ARE_957523_309003212_1420_21032016_5.activations'),Path('activations/valid/agravo_em_recurso_extraordinario/ARE_811832_4448371_1420_18052014_17.activations'),Path('activations/valid/agravo_em_recurso_extraordinario/ARE_1053618_311977154_1420_07062017_63.activations'),Path('activations/valid/agravo_em_recurso_extraordinario/ARE_1066591_312429859_1420_11082017_9.activations'),Path('activations/valid/agravo_em_recurso_extraordinario/ARE_1071533_312604954_1420_30082017_6.activations'),Path('activations/valid/agravo_em_recurso_extraordinario/ARE_1105199_313609724_1420_04022018_13.activations'),Path('activations/valid/agravo_em_recurso_extraordinario/ARE_1010513_310734552_1420_07022017_5.activations'),Path('activations/valid/agravo_em_recurso_extraordinario/ARE_1056724_312077006_1420_27062017_10.activations'),Path('activations/valid/agravo_em_recurso_extraordinario/ARE_1115582_313936746_1420_16032018_5.activations'),Path('ac

In [6]:
class TensorToTorch(Transform):
    def encodes(self, x):
        with open(x, "rb") as file:
            t = pickle.load(file)
        return tensor(np.array(t)).view(-1)

In [7]:
tfms = [TensorToTorch(),[parent_label, Categorize()]]
splits = GrandparentSplitter(valid_name='valid')(files)
dsets = Datasets(files, tfms, splits=splits)
dls = dsets.dataloaders()

In [8]:
dls.bs

64

In [18]:
dls.train_ds

(#158308) [(tensor([0.8886, 0.0510, 0.0319, 0.0047, 0.0214, 0.0023, 0.8886, 0.0510, 0.0319,
        0.0047, 0.0214, 0.0023]), TensorCategory(1)),(tensor([0.7876, 0.1109, 0.0576, 0.0051, 0.0336, 0.0053, 0.7876, 0.1109, 0.0576,
        0.0051, 0.0336, 0.0053]), TensorCategory(1)),(tensor([0.8681, 0.0575, 0.0409, 0.0046, 0.0260, 0.0030, 0.8681, 0.0575, 0.0409,
        0.0046, 0.0260, 0.0030]), TensorCategory(1)),(tensor([0.9497, 0.0156, 0.0154, 0.0050, 0.0132, 0.0011, 0.9497, 0.0156, 0.0154,
        0.0050, 0.0132, 0.0011]), TensorCategory(1)),(tensor([0.9470, 0.0215, 0.0138, 0.0033, 0.0132, 0.0012, 0.9470, 0.0215, 0.0138,
        0.0033, 0.0132, 0.0012]), TensorCategory(1)),(tensor([0.8549, 0.0694, 0.0416, 0.0047, 0.0260, 0.0033, 0.8549, 0.0694, 0.0416,
        0.0047, 0.0260, 0.0033]), TensorCategory(1)),(tensor([0.9342, 0.0288, 0.0177, 0.0041, 0.0138, 0.0014, 0.9342, 0.0288, 0.0177,
        0.0041, 0.0138, 0.0014]), TensorCategory(1)),(tensor([0.7947, 0.1064, 0.0623, 0.0055, 0.0261, 0.

In [ ]:
test_splits = GrandparentSplitter(valid_name='test')(files)
test_dsets = Datasets(files, tfms, splits=test_splits)
test_dls = test_dsets.dataloaders()

In [ ]:
class ImgTextFusion(Module):
    def __init__(self, head):
        self.head = head
    
    def forward(self, x):
        return self.head(x)

In [ ]:
def create_head(nf, n_out, lin_ftrs=None, ps=0.5, bn_final=False, lin_first=False):
    "Model head that takes `nf` features, runs through `lin_ftrs`, and out `n_out` classes."
    lin_ftrs = [nf, 512, n_out] if lin_ftrs is None else [nf] + lin_ftrs + [n_out]
    ps = L(ps)
    if len(ps) == 1: ps = [ps[0]/2] * (len(lin_ftrs)-2) + ps
    actns = [nn.ReLU(inplace=True)] * (len(lin_ftrs)-2) + [None]
    layers = []
    if lin_first: layers.append(nn.Dropout(ps.pop(0)))
    for ni,no,p,actn in zip(lin_ftrs[:-1], lin_ftrs[1:], ps, actns):
        layers += LinBnDrop(ni, no, bn=True, p=p, act=actn, lin_first=lin_first)
    if lin_first: layers.append(nn.Linear(lin_ftrs[-2], n_out))
    if bn_final: layers.append(nn.BatchNorm1d(lin_ftrs[-1], momentum=0.01))
    return nn.Sequential(*layers)

In [ ]:
head = create_head(12, OUT_DIM)

In [ ]:
model = ImgTextFusion(head)

In [ ]:
model

In [ ]:
def loss_func(out, targ):
    return CrossEntropyLossFlat()(out, targ.long())

In [ ]:
fscore = F1Score(average='weighted')

In [ ]:
learn = Learner(dls, model, loss_func=loss_func, metrics=[accuracy, fscore])

In [ ]:
dls.one_batch()[0].shape

In [ ]:
learn.model(dls.one_batch()[0])

In [ ]:
learn.lr_find()

In [ ]:
learn.fit_one_cycle(10, 3e-3)